In [1]:
import cv2
import time
import numpy as np
from random import randint
import sys, os
from sys import platform
import argparse
from math import sqrt, acos, degrees, atan, degrees
import numpy as np
import random
import glob
from PIL import Image
import itertools
import _pickle as pickle
from datetime import datetime

In [2]:
protoFile = "pose/coco/pose_deploy_linevec.prototxt.txt"
weightsFile = "pose/coco/pose_iter_440000.caffemodel"
nPoints = 18
# COCO Output Format
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']

POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16] ]

# index of pafs correspoding to the POSE_PAIRS
# e.g for POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output, Similarly, (1,5) -> (39,40) and so on.
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44],
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30],
          [47,48], [49,50], [53,54], [51,52], [55,56],
          [37,38], [45,46]]

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]



# In[78]:


def process_single_image(image1):
    
    def getKeypoints(probMap, threshold=0.1):

        mapSmooth = cv2.GaussianBlur(probMap,(3,3),0,0)

        mapMask = np.uint8(mapSmooth>threshold)
        keypoints_list = []
        keypoints_array = np.zeros((1,4))

        #find the blobs
        contours,b = cv2.findContours(mapMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        num_joints_detected = 0
        ss = []

        #for each blob find the maxima
        for cnt in contours:
            num_joints_detected = num_joints_detected + 1
            blobMask = np.zeros(mapMask.shape)
            blobMask = cv2.fillConvexPoly(blobMask, cnt, 1)
            maskedProbMap = mapSmooth * blobMask
            _, maxVal, _, maxLoc = cv2.minMaxLoc(maskedProbMap)
#             if res == 0:
#                 print('bvvvvvvvvvvvvvv')
#                 print(maxLoc)
#                 maxLoc = (0,0)
#                 a = (0,)              
#             else:          
#                 print('cvvvvvvvvvvvvvv')
#                 a = (probMap[maxLoc[1], maxLoc[0]],)                                
            ss.append(maxLoc[0])
            ss.append(maxLoc[1])
            keypoints_list.append(maxLoc + (probMap[maxLoc[1], maxLoc[0]],))
            #keypoints.append(maxLoc + a)
            
        return keypoints_list, num_joints_detected, ss


    # Find valid connections between the different joints of a all persons present
    def getValidPairs(output):
        valid_pairs = []
        invalid_pairs = []
        n_interp_samples = 10
        paf_score_th = 0.1
        conf_th = 0.7
        # loop for every POSE_PAIR
        for k in range(len(mapIdx)):
            # A->B constitute a limb
            pafA = output[0, mapIdx[k][0], :, :]
            pafB = output[0, mapIdx[k][1], :, :]
            pafA = cv2.resize(pafA, (frameWidth, frameHeight))
            pafB = cv2.resize(pafB, (frameWidth, frameHeight))

            # Find the keypoints for the first and second limb
            candA = detected_keypoints[POSE_PAIRS[k][0]]
            candB = detected_keypoints[POSE_PAIRS[k][1]]
            nA = len(candA)
            nB = len(candB)

            # If keypoints for the joint-pair is detected
            # check every joint in candA with every joint in candB
            # Calculate the distance vector between the two joints
            # Find the PAF values at a set of interpolated points between the joints
            # Use the above formula to compute a score to mark the connection valid

            if( nA != 0 and nB != 0):
                valid_pair = np.zeros((0,3))
                for i in range(nA):
                    max_j=-1
                    maxScore = -1
                    found = 0
                    for j in range(nB):
                        # Find d_ij
                        d_ij = np.subtract(candB[j][:2], candA[i][:2])
                        norm = np.linalg.norm(d_ij)
                        if norm:
                            d_ij = d_ij / norm
                        else:
                            continue
                        # Find p(u)
                        interp_coord = list(zip(np.linspace(candA[i][0], candB[j][0], num=n_interp_samples),
                                                np.linspace(candA[i][1], candB[j][1], num=n_interp_samples)))
                        # Find L(p(u))
                        paf_interp = []
                        for k in range(len(interp_coord)):
                            paf_interp.append([pafA[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))],
                                               pafB[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))] ])
                        # Find E
                        paf_scores = np.dot(paf_interp, d_ij)
                        avg_paf_score = sum(paf_scores)/len(paf_scores)

                        # Check if the connection is valid
                        # If the fraction of interpolated vectors aligned with PAF is higher then threshold -> Valid Pair
                        if ( len(np.where(paf_scores > paf_score_th)[0]) / n_interp_samples ) > conf_th :
                            if avg_paf_score > maxScore:
                                max_j = j
                                maxScore = avg_paf_score
                                found = 1
                    # Append the connection to the list
                    if found:
                        valid_pair = np.append(valid_pair, [[candA[i][3], candB[max_j][3], maxScore]], axis=0)

                # Append the detected connections to the global list
                valid_pairs.append(valid_pair)
            else: # If no keypoints are detected
                #print("No Connection : k = {}".format(k))
                invalid_pairs.append(k)
                valid_pairs.append([])
        return valid_pairs, invalid_pairs


    # This function creates a list of keypoints belonging to each person
    # For each detected valid pair, it assigns the joint(s) to a person
    def getPersonwiseKeypoints(valid_pairs, invalid_pairs):
        # the last number in each row is the overall score
        personwiseKeypoints = -1 * np.ones((0, 19))

        for k in range(len(mapIdx)):
            if k not in invalid_pairs:
                partAs = valid_pairs[k][:,0]
                partBs = valid_pairs[k][:,1]
                indexA, indexB = np.array(POSE_PAIRS[k])

                for i in range(len(valid_pairs[k])):
                    found = 0
                    person_idx = -1
                    for j in range(len(personwiseKeypoints)):
                        if personwiseKeypoints[j][indexA] == partAs[i]:
                            person_idx = j
                            found = 1
                            break

                    if found:
                        personwiseKeypoints[person_idx][indexB] = partBs[i]
                        personwiseKeypoints[person_idx][-1] += keypoints_list[partBs[i].astype(int), 2] + valid_pairs[k][i][2]

                    # if find no partA in the subset, create a new subset
                    elif not found and k < 17:
                        row = -1 * np.ones(19)
                        row[indexA] = partAs[i]
                        row[indexB] = partBs[i]
                        # add the keypoint_scores for the two keypoints and the paf_score
                        row[-1] = sum(keypoints_list[valid_pairs[k][i,:2].astype(int), 2]) + valid_pairs[k][i][2]
                        personwiseKeypoints = np.vstack([personwiseKeypoints, row])
        return personwiseKeypoints

    frameWidth = image1.shape[1]
    frameHeight = image1.shape[0]
    t = time.time()
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

    # Fix the input Height and get the width according to the Aspect Ratio

    inpBlob = cv2.dnn.blobFromImage(image1, 1.0 / 255, (256, 256),
                              (0, 0, 0), swapRB=False, crop=False)
    kp = np.zeros((frameWidth, frameHeight))
    net.setInput(inpBlob)
    output = net.forward()
    #print("Time Taken in forward pass = {}".format(time.time() - t))
    detected_keypoints = []
    keypoints_list = np.zeros((0,3))
    keypoint_id = 0
    threshold = 0.1
    num_joints_detected_list = []
    list2 = []

    for part in range(nPoints):
        probMap = output[0,part,:,:]
        probMap = cv2.resize(probMap, (image1.shape[1], image1.shape[0]))
        keypoints, ttt, ss = getKeypoints(probMap, threshold)
        list2.append(np.asarray(ss))
        num_joints_detected_list.append(ttt)
        #print("Keypoints - {} : {}".format(keypointsMapping[part], keypoints))
        keypoints_with_id = []

        for i in range(len(keypoints)):
            keypoints_with_id.append(keypoints[i] + (keypoint_id,))
            keypoints_list = np.vstack([keypoints_list, keypoints[i]])
            keypoint_id += 1

        detected_keypoints.append(keypoints_with_id)

#     print("key_points list:")
#     print(keypoints_list)
    num_people = max(num_joints_detected_list)
    keypoints_array = np.zeros((18,2*num_people))
    for i in range(len(num_joints_detected_list)):
        for j in range(2*num_joints_detected_list[i]):
            keypoints_array[i][j] = list2[i][j]
#     print("Keypoints - {} ".format(keypoints_array))
#     print("Shape of keypoints: {}".format(keypoints_array.shape))
    composite = image1.copy()
    for i in range(nPoints):
        for j in range(len(detected_keypoints[i])):
            cv2.circle(kp, detected_keypoints[i][j][0:2], 3, (255, 255, 255), -1, cv2.LINE_AA)
            cv2.circle(composite, detected_keypoints[i][j][0:2], 3, (255, 255, 255), -1, cv2.LINE_AA)

    #print("-------------------------------------------------------------------")
    valid_pairs, invalid_pairs = getValidPairs(output)
    personwiseKeypoints = getPersonwiseKeypoints(valid_pairs, invalid_pairs)
    skeleton = kp.copy()
    for i in range(17):
        for n in range(len(personwiseKeypoints)):
            index = personwiseKeypoints[n][np.array(POSE_PAIRS[i])]
            if -1 in index:
                continue
            B = np.int32(keypoints_list[index.astype(int), 0])
            A = np.int32(keypoints_list[index.astype(int), 1])
            cv2.line(skeleton, (B[0], A[0]), (B[1], A[1]), (255, 255, 255), 2, cv2.LINE_AA)
        
    return kp, composite, skeleton, keypoints_array

In [4]:
start=datetime.now()
kinect = "kinect_k01"


data = []
repeat1 = ['S01', 'S02', 'S03', 'S04', 'S05','S06', 'S07', 'S08', 'S09', 'S10', 'S11', 'S12']
repeat2 = ['A01', 'A02', 'A03', 'A04', 'A05','A06', 'A07', 'A08', 'A09', 'A10', 'A11']
repeat3 = ['R01', 'R02', 'R03', 'R04', 'R05']

# repeat1 = ['S03']
# repeat2 = ['A04']
# repeat3 = ['R01', 'R02', 'R03', 'R04', 'R05']
labels = []
for aa in range(len(repeat1)): # Loop at subject level
    for ab in range (len(repeat2)): # Loop at Action level
        action_label = np.array([int(repeat2[ab][1:3])])
        action_bael.shape=[1,1]
        for b in range (len(repeat3)): # Loop at repetition level
            path = os.getcwd() + "/Kin01/" + str(repeat1[aa]) + "/" +  str(repeat2[ab]) +  "/" + str(repeat3[b])
            rep = str(repeat3[b])
            if str(repeat1[aa])=='S04' and str(repeat2[ab])=='A08'  and str(repeat3[b])=='R05': # Because this        
                continue
            else:
                dirs = os.listdir(path)
                if not os.path.exists(path+'/resized_images'):
                    os.mkdir(path+'/resized_images')
                    os.mkdir(path+'/composite_images')
                    os.mkdir(path+'/skeleton_images')
                final_size = 256
                j = 0
                print ("Path:"+str(path))
                for item in dirs: # Loop through all images of a repetition
                    t = path + "/" + item
                    if os.path.isfile(t):  
                        im = Image.open(t)
                        f,e = os.path.splitext(t)
                        size = im.size
                        ratio = float(final_size) / max(size)
                        new_image_size = tuple([int(x * ratio) for x in size])
                        im = im.resize(new_image_size, Image.ANTIALIAS)
#                         print(im.size)
                        new_im = Image.new("RGB", (final_size, final_size))
#                         print(new_im.size)
                        new_im.paste(im, ((final_size - new_image_size[0]) // 2, (final_size - new_image_size[1]) // 2))
                        image1 = cv2.cvtColor(np.asarray(new_im), cv2.COLOR_RGB2BGR)
                        filename = os.path.join(path + '/resized_images/'+repeat1[aa]+'_'+repeat2[ab]+'_'+repeat3[b]+'_'+str(j+1)+'.jpg')
                        new_im.save(filename, 'JPEG', quality=90)

                        img1 = cv2.imread(path + '/resized_images/'+repeat1[aa]+'_'+repeat2[ab]+'_'+repeat3[b]+'_'+str(j+1)+'.jpg',0)
                        img1 = cv2.resize(img1, (img_vector_size, img_vector_size), interpolation=cv2.INTER_AREA)

                        img_label = np.array([j+1])
                        img_label.shape = [1,1]
                        resize_vector = np.concatenate((img1.reshape(1, img_vector_size*img_vector_size), img_label, action_label), axis = 1)

                        if j==0:
                            resize_array = resize_vector
                        else:
                            resize_array = np.concatenate((resize_array, resize_vector), axis = 0)

                        #print(image1.shape)

                        kp, composite, skeleton, keypoints_array = process_single_image(image1)
                        num.append(int((keypoints_array.shape[1])/2))
                        kp = cv2.resize(kp, (img_vector_size, img_vector_size), interpolation=cv2.INTER_AREA)
                        composite = cv2.resize(composite, (img_vector_size, img_vector_size), interpolation=cv2.INTER_AREA)
                        skeleton = cv2.resize(skeleton, (img_vector_size, img_vector_size), interpolation=cv2.INTER_AREA)
                        cv2.imwrite(path + '/skeleton_images/'+repeat1[aa]+'_'+repeat2[ab]+'_'+repeat3[b]+'_'+str(j+1)+'.jpg',skeleton)
                        cv2.imwrite(path + '/composite_images/'+repeat1[aa]+'_'+repeat2[ab]+'_'+repeat3[b]+'_'+str(j+1)+'.jpg',composite)
            #             cv2.imwrite(path + '/keypoint_images/'+str(kinect)+'_'+str(subject)+'_'+str(action)+'_'+str(rep)+'_ ('+str(j+1)+').jpg',kp)

                        composite = cv2.cvtColor(composite, cv2.COLOR_BGR2GRAY)

                        skeleton_vector = np.concatenate((skeleton.reshape(1,img_vector_size*img_vector_size), img_label, action_label), axis = 1)
                        if j==0:
                            skeleton_array = skeleton_vector
                        else:
                            skeleton_array = np.concatenate((skeleton_array, skeleton_vector), axis = 0)

                        composite_vector = np.concatenate((composite.reshape(1,img_vector_size*img_vector_size), img_label, action_label), axis = 1)
                        if j==0:
                            composite_array = composite_vector
                        else:
                            composite_array = np.concatenate((composite_array, composite_vector), axis = 0)
                        kp1 = keypoints_array.reshape(1,keypoints_array.shape[0]*keypoints_array.shape[1])
                        print("kp1 Shape: {}".format(kp1.shape))
                        if kp1.shape[1] == 0:
                            kp2 = np.zeros((1,36))
                            kp2 = kp2.reshape(1,36)
                        else:
                            kp2 = np.zeros((1,36))
                            kp2 = kp1[0,0:36]
                            kp2 = kp2.reshape(1,36)

                        print("kp2 Shape: {}".format(kp2.shape))
                        kp_vector = np.concatenate((kp2, img_label, action_label), axis = 1)
                        if j == 0:
                            kp_arr = kp_vector
                        else:
                            kp_arr = np.concatenate((kp_arr, kp_vector), axis=0)
                        print("image: "+str(j+1))
                        print(kp_arr.shape)
                        j = j+1
                np.savetxt(repeat1[aa]+'_'+repeat2[ab]+'_'+repeat3[b]+'_resized_images.csv',resize_array, fmt='%5.8g', delimiter = ',')
                np.savetxt(repeat1[aa]+'_'+repeat2[ab]+'_'+repeat3[b]+'_composite_images.csv',composite_array, fmt='%5.8g', delimiter = ',')
                np.savetxt(repeat1[aa]+'_'+repeat2[ab]+'_'+repeat3[b]+'_skeleton_images.csv',skeleton_array, fmt='%5.8g', delimiter = ',')
                np.savetxt(repeat1[aa]+'_'+repeat2[ab]+'_'+repeat3[b]+'_keypoint_locations.csv', kp_arr, fmt='%5.8g', delimiter = ',')
#     #max_people = max(num)
#     print("Number of images: {}".format(j))
# print("Maximum number of people: "+str(max_people))
# kps_array = np.zeros((j,36*max_people+2))
# for a in range (j):
# 	kp_v = list_kp_vectors[a]
# 	for b in range(kp_v.shape[1]-2):
# 		kps_array[a,b] = kp_v[0,b]
# 		kps_array[a,-2] = kp_v[0,-2]
# 		kps_array[a,-1] = kp_v[0,-1]
print("Time taken: {}".format(datetime.now()-start)) 

F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A01/R01
R01
123
123
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A01/R02
R02
243
243
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A01/R03
R03
385
385
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A01/R04
R04
512
512
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A01/R05
R05
640
640
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A02/R01
R01
810
810
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A02/R02
R02
984
984
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A02/R03
R03
1162
1162
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A02/R04
R04
1331
1331
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A02/R05
R05
1502
1502
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A03/R01
R01
1854
1854
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A03/R02
R02
2217
2217
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A03/R03
R03
2612
2612
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A03/R04
R04
2983
2983
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A03/R05
R05
3365
3365
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S04/A04/R01
R01


23350
23350
F:\datasets\BerkeleyMHAD\Kinect/Kin01/S06/A03/R05
R05


OSError: image file is truncated (1065 bytes not processed)